In [2]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

In [3]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [4]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [5]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        for mrn in cohort_indexes: 
            x = get_mrn_has_certain_condition_WINDOW(mrn,cohort_condition,df_mrn.loc[mrn]["HT_Onset"]-gap_in_days)
            a = [mrn, x]
            cohort_mrn_diagnosis.append(a)
    else:
        print("function not available")
    #convert list cohort_mrn_diagnosis to panda
    #condition_name= 'has_condition_'+name_feature
    #col_names = ['medical_record_number',condition_name]
    #df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    #df_final.set_index('medical_record_number', inplace=True)
    #df_final=df_mrn.merge( df_final, left_index=True, right_index=True)
    #df_final.to_parquet(name_df)
    #print(df_final)
    df_final = cohort_mrn_diagnosis
    return df_final

In [7]:
condition_oCV = (Diagnosis("420%", "ICD-9")| Diagnosis("I30%", "ICD-10") | 
                Diagnosis("I31%", "ICD-10")| Diagnosis("I32%", "ICD-10") |
                Diagnosis("I33%", "ICD-10")| Diagnosis("I34%", "ICD-10") |
                Diagnosis("I35%", "ICD-10")| Diagnosis("421%", "ICD-9") |
                Diagnosis("422%", "ICD-9")| Diagnosis("423%", "ICD-9") |
                Diagnosis("424%", "ICD-9") | Diagnosis("425%", "ICD-9") |
                Diagnosis("426%", "ICD-9") | Diagnosis("427%", "ICD-9") |
                Diagnosis("428%", "ICD-9") | Diagnosis("429%", "ICD-9") |
                Diagnosis("428%", "ICD-9") | Diagnosis("429%", "ICD-9") |
                Diagnosis("I36%", "ICD-10")| Diagnosis("I37%", "ICD-10") |
                Diagnosis("I38%", "ICD-10")| Diagnosis("I39%", "ICD-10") |
                Diagnosis("I40%", "ICD-10")| Diagnosis("I41%", "ICD-10") |
                Diagnosis("I42%", "ICD-10")| Diagnosis("I43%", "ICD-10") |
                Diagnosis("I44%", "ICD-10")| Diagnosis("I45%", "ICD-10") |
                Diagnosis("I46%", "ICD-10")| Diagnosis("I47%", "ICD-10") |
                Diagnosis("I48%", "ICD-10")| Diagnosis("I49%", "ICD-10") |
                Diagnosis("I50%", "ICD-10")| Diagnosis("I51%", "ICD-10") |
                Diagnosis("I52%", "ICD-10")
                )

#ICD 9 (420-429), ICD 10 (I30-I52)

In [8]:
oCV_COUNT = get_has_certain_condition(condition_oCV , Case_filtered_15_540, "other_form_heart_disease", 180 , "COUNT", "Case")


Fetching data for Diagnosis (...)


In [10]:
oCV_COUNT.loc[oCV_COUNT['number_of_occurences_other_form_heart_disease'] != 0]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_other_form_heart_disease
medical_record_number,,,,,,,,,,,,,
1010016947,32,24285.0,23563.0,24088.0,23563.0,Age_ICD_condition,22709,854.0,1952,Other,Pt Declined,Female,2.0
1022736112,80,25425.0,25388.0,NaN,25388.0,Age_ICD_condition,23192,2196.0,1944,White,Unknown,Male,1.0
1060593816,52,24648.0,NaN,24404.0,24404.0,Age_MED_condition,21345,3059.0,1950,White,Unknown,Female,14.0
1061539865,77,NaN,22584.0,22584.0,22584.0,Age_ICD_condition,21095,1489.0,1943,White,Jewish,Female,1.0
1080190855,104,20550.0,20550.0,20550.0,20550.0,Age_BP_condition,20010,540.0,1955,White,,Male,3.0
1153304445,191,18765.0,18795.0,18795.0,18765.0,Age_BP_condition,17331,1434.0,1961,White,PT Declined,Male,2.0
1201995691,43,31251.0,NaN,31251.0,31251.0,Age_BP_condition,30536,715.0,1932,African-American,Baptist,Female,2.0
120727686,294,24961.0,24790.0,24790.0,24790.0,Age_ICD_condition,23209,1581.0,1938,Other,Christian,Male,5.0
1330685947,67,NaN,4606.0,4607.0,4606.0,Age_ICD_condition,4062,544.0,1998,Unknown,Other,Female,2.0


In [11]:
oCV_EVER = get_has_certain_condition(condition_oCV , Case_filtered_15_540, "other_form_heart_disease", 180 , "EVER", "Case")


Fetching data for Diagnosis (...)


In [13]:
oCV_EVER.loc[oCV_EVER['has_condition_other_form_heart_disease'] == 1]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_other_form_heart_disease
medical_record_number,,,,,,,,,,,,,
1010016947,32,24285.0,23563.0,24088.0,23563.0,Age_ICD_condition,22709,854.0,1952,Other,Pt Declined,Female,1.0
1022736112,80,25425.0,25388.0,NaN,25388.0,Age_ICD_condition,23192,2196.0,1944,White,Unknown,Male,1.0
1060593816,52,24648.0,NaN,24404.0,24404.0,Age_MED_condition,21345,3059.0,1950,White,Unknown,Female,1.0
1061539865,77,NaN,22584.0,22584.0,22584.0,Age_ICD_condition,21095,1489.0,1943,White,Jewish,Female,1.0
1080190855,104,20550.0,20550.0,20550.0,20550.0,Age_BP_condition,20010,540.0,1955,White,,Male,1.0
1153304445,191,18765.0,18795.0,18795.0,18765.0,Age_BP_condition,17331,1434.0,1961,White,PT Declined,Male,1.0
1201995691,43,31251.0,NaN,31251.0,31251.0,Age_BP_condition,30536,715.0,1932,African-American,Baptist,Female,1.0
120727686,294,24961.0,24790.0,24790.0,24790.0,Age_ICD_condition,23209,1581.0,1938,Other,Christian,Male,1.0
1330685947,67,NaN,4606.0,4607.0,4606.0,Age_ICD_condition,4062,544.0,1998,Unknown,Other,Female,1.0


In [14]:
Other_form_heart_disease_Case = oCV_EVER.merge(oCV_COUNT['number_of_occurences_other_form_heart_disease'], left_index=True, right_index=True)


In [15]:
Other_form_heart_disease_Case

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_other_form_heart_disease,number_of_occurences_other_form_heart_disease
medical_record_number,,,,,,,,,,,,,,
100067499,82,24103.0,NaN,24106.0,24103.0,Age_BP_condition,21049,3054.0,1951,White,Other,Male,0.0,0.0
100091035,154,11369.0,12938.0,12881.0,11369.0,Age_BP_condition,9141,2228.0,1978,Ba,Other,Male,0.0,0.0
1004105957,43,22961.0,NaN,24104.0,22961.0,Age_BP_condition,19914,3047.0,1952,White,Greek Orthodox,Male,0.0,0.0
100423963,20,NaN,23584.0,23584.0,23584.0,Age_ICD_condition,19827,3757.0,1945,Other,Pt Declined,Male,0.0,0.0
1005188482,279,20636.0,20514.0,20514.0,20514.0,Age_ICD_condition,19884,630.0,1958,Ba,Jewish,Male,0.0,0.0
1006954100,28,15242.0,15536.0,15536.0,15242.0,Age_BP_condition,13828,1414.0,1973,Unknown,Catholic,Female,0.0,0.0
1006987597,72,27618.0,27618.0,27618.0,27618.0,Age_BP_condition,26706,912.0,1941,White,Catholic,Male,0.0,0.0
1007749798,93,NaN,27233.0,27233.0,27233.0,Age_ICD_condition,25235,1998.0,1939,White,Catholic,Female,0.0,0.0
1007977434,221,25355.0,24347.0,24347.0,24347.0,Age_ICD_condition,23605,742.0,1943,White,Unknown,Male,0.0,0.0


In [16]:
Other_form_heart_disease_Case.to_parquet("Other_form_heart_disease_Case.parquet")

In [ ]:
##### Controls ####

In [17]:
oCV_COUNT = get_has_certain_condition(condition_oCV , Control_filtered_15_540, "Other_form_heart_disease", 180 , "COUNT", "Control")

Fetching data for Diagnosis (...)


In [18]:
oCV_EVER = get_has_certain_condition(condition_oCV, Control_filtered_15_540, "Other_form_heart_disease", 180 , "EVER", "Control")


Fetching data for Diagnosis (...)


In [20]:
oCV_EVER.loc[oCV_EVER['has_condition_Other_form_heart_disease'] == 1]

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Other_form_heart_disease
medical_record_number,,,,,,,,,
100064127,21186,32,19250,1936,1960,Unknown,Unknown,Female,1.0
1000829189,25648,30,22684,2964,1946,Unknown,Unknown,Male,1.0
1000925805,23372,79,21264,2108,1954,White,None,Female,1.0
1001143259,19179,47,15799,3380,1965,White,Unknown,Female,1.0
1001498353,24356,78,20133,4223,1951,White,PT Declined,Female,1.0
10015923,10670,32,7848,2822,1985,White,Jewish,Female,1.0
1002393812,19283,105,14903,4380,1965,White,Jewish,Female,1.0
1002445620,12665,23,9502,3163,1978,Other,Unknown,Female,1.0
1002646579,20873,75,18444,2429,1961,Unknown,,Female,1.0


In [21]:
Other_form_heart_disease_Control = oCV_EVER.merge(oCV_COUNT['number_of_occurences_Other_form_heart_disease'], left_index=True, right_index=True)


In [22]:
Other_form_heart_disease_Control

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Other_form_heart_disease,number_of_occurences_Other_form_heart_disease
medical_record_number,,,,,,,,,,
100002558,21414,15,19951,1463,1959,White,Other,Female,0.0,0.0
100002884,23929,44,18526,5403,1952,Unknown,,Male,0.0,0.0
1000068212,10116,29,9228,888,1990,White,Unknown,Female,0.0,0.0
1000083464,25630,20,23026,2604,1948,White,Catholic,Male,0.0,0.0
1000119948,17087,163,15757,1330,1971,African-American,None,Male,0.0,0.0
1000165222,17506,627,12215,5291,1970,Other,Catholic,Female,0.0,0.0
1000199143,26941,141,22805,4136,1944,White,Jewish,Female,0.0,0.0
1000205924,14867,108,10618,4249,1976,Unknown,Unknown,Female,0.0,0.0
1000212584,19220,15,16818,2402,1965,White,,Male,0.0,0.0


In [23]:
Other_form_heart_disease_Control.to_parquet("Other_form_heart_disease_Control.parquet")